In [42]:
import requests
import gzip
import shutil
import pandas as pd
from geopy.distance import geodesic

## Data Upload

### Tokyo Q3 Data Upload

In [14]:
url = "https://github.com/HDuo1129/Data-Analysis-3/raw/main/Assignment%202/Dataset/Tokyo/2024%20Q3/listings.csv.gz"
file_path = "listings.csv.gz"

response = requests.get(url)
with open(file_path, "wb") as f:
    f.write(response.content)

with gzip.open("listings.csv.gz", "rb") as f_in:
    with open("listings.csv", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [15]:
Tokyo_Q3 = pd.read_csv("listings.csv.gz", compression="gzip")
print(Tokyo_Q3.head())

        id                           listing_url       scrape_id last_scraped  \
0   197677   https://www.airbnb.com/rooms/197677  20240927024131   2024-09-27   
1   776070   https://www.airbnb.com/rooms/776070  20240927024131   2024-09-27   
2   905944   https://www.airbnb.com/rooms/905944  20240927024131   2024-09-27   
3  1016831  https://www.airbnb.com/rooms/1016831  20240927024131   2024-09-27   
4  1196177  https://www.airbnb.com/rooms/1196177  20240927024131   2024-09-27   

        source                                               name  \
0  city scrape                          Oshiage Holiday Apartment   
1  city scrape                             Kero-kero house room 1   
2  city scrape  4F Spacious Apartment in Shinjuku / Shibuya Tokyo   
3  city scrape         5 mins Shibuya Cat modern sunny  Shimokita   
4  city scrape        Stay with host Cozy private room Senju area   

                                         description  \
0                                         

### Tokyo Q4 Data Upload

In [42]:
url = "https://github.com/HDuo1129/Data-Analysis-3/raw/main/Assignment%202/Dataset/Tokyo/2024%20Q4/listings.csv.gz"
file_path = "listings.csv.gz"

response = requests.get(url)
with open(file_path, "wb") as f:
    f.write(response.content)

with gzip.open("listings.csv.gz", "rb") as f_in:
    with open("listings.csv", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [44]:
Tokyo_Q4 = pd.read_csv("listings.csv.gz", compression="gzip")
print(Tokyo_Q4.head())

        id                           listing_url       scrape_id last_scraped  \
0   197677   https://www.airbnb.com/rooms/197677  20241230011552   2024-12-30   
1   776070   https://www.airbnb.com/rooms/776070  20241230011552   2024-12-30   
2   905944   https://www.airbnb.com/rooms/905944  20241230011552   2024-12-30   
3  1016831  https://www.airbnb.com/rooms/1016831  20241230011552   2024-12-30   
4  1196177  https://www.airbnb.com/rooms/1196177  20241230011552   2024-12-30   

        source                                               name  \
0  city scrape                          Oshiage Holiday Apartment   
1  city scrape                             Kero-kero house room 1   
2  city scrape  4F Spacious Apartment in Shinjuku / Shibuya Tokyo   
3  city scrape         5 mins Shibuya Cat modern sunny  Shimokita   
4  city scrape   Homestay at Host's House -  Senju-Ohashi Station   

                                         description  \
0                                         

### Bangkok Q3 Data Upload

In [46]:
url = "https://github.com/HDuo1129/Data-Analysis-3/raw/main/Assignment%202/Dataset/Bangkok/2024%20Q3/listings.csv.gz"
file_path = "listings.csv.gz"

response = requests.get(url)
with open(file_path, "wb") as f:
    f.write(response.content)

with gzip.open("listings.csv.gz", "rb") as f_in:
    with open("listings.csv", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [48]:
Bangkok_Q3 = pd.read_csv("listings.csv.gz", compression="gzip")
print(Bangkok_Q3.head())

      id                         listing_url       scrape_id last_scraped  \
0  27934  https://www.airbnb.com/rooms/27934  20240925020907   2024-09-25   
1  27979  https://www.airbnb.com/rooms/27979  20240925020907   2024-09-25   
2  28745  https://www.airbnb.com/rooms/28745  20240925020907   2024-09-25   
3  35780  https://www.airbnb.com/rooms/35780  20240925020907   2024-09-25   
4  48736  https://www.airbnb.com/rooms/48736  20240925020907   2024-09-25   

            source                                            name  \
0      city scrape                 Nice room with superb city view   
1  previous scrape                  Easy going landlord,easy place   
2  previous scrape               modern-style apartment in Bangkok   
3      city scrape  Spacious one bedroom at The Kris Condo Bldg. 3   
4      city scrape                 Condo with Chaopraya River View   

                                         description  \
0  Our cool and comfortable one bedroom apartment...   
1   

## Data Cleaning

### Tokyo Q3 Data Cleaning

In [45]:
columns_to_drop = [
    "scrape_id", "source", "neighbourhood_group_cleansed", "calendar_updated", "listing_url", "picture_url", 
    "host_url", "host_thumbnail_url", "host_picture_url", "host_neighbourhood", "neighbourhood", "calendar_last_scraped",
    "minimum_minimum_nights", "maximum_minimum_nights", "minimum_maximum_nights", "maximum_maximum_nights", 
    "minimum_nights_avg_ntm", "maximum_nights_avg_ntm", "calculated_host_listings_count_entire_homes", 
    "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms"]
Tokyo_Q3_cleaned = Tokyo_Q3.drop(columns=columns_to_drop)

fillna_dict = {
    "description": "",
    "neighborhood_overview": "",
    "host_about": "",
    "host_location": "Unknown",
    "host_response_time": "Unknown",
    "host_response_rate": "0%",
    "host_acceptance_rate": "0%",
    "bathrooms": Tokyo_Q3_cleaned["bathrooms"].median(),
    "bedrooms": Tokyo_Q3_cleaned["bedrooms"].median(),
    "beds": Tokyo_Q3_cleaned["beds"].median(),
    "review_scores_rating": 0,
    "review_scores_accuracy": 0,
    "review_scores_cleanliness": 0,
    "review_scores_checkin": 0,
    "review_scores_communication": 0,
    "review_scores_location": 0,
    "review_scores_value": 0,
    "reviews_per_month": 0}
Tokyo_Q3_cleaned.fillna(fillna_dict, inplace=True)

In [47]:
# Price Check
Tokyo_Q3_cleaned["price"] = (
    Tokyo_Q3_cleaned["price"]
    .astype(str)
    .str.replace("$", "")
    .str.replace(",", "")
    .astype(float))

# Rate Check
Tokyo_Q3_cleaned["host_response_rate"] = (
    Tokyo_Q3_cleaned["host_response_rate"]
    .str.replace("%", "")
    .astype(float))
Tokyo_Q3_cleaned["host_acceptance_rate"] = (
    Tokyo_Q3_cleaned["host_acceptance_rate"]
    .str.replace("%", "")
    .astype(float))

# Date Check
date_cols = ["host_since", "first_review", "last_review"]
for col in date_cols:
    Tokyo_Q3_cleaned[col] = pd.to_datetime(Tokyo_Q3_cleaned[col], errors="coerce")

# TF Result Check
bool_cols = ["instant_bookable", "host_is_superhost", "host_identity_verified"]
for col in bool_cols:
    Tokyo_Q3_cleaned[col] = Tokyo_Q3_cleaned[col].map({"t": True, "f": False})

# Calculate Landlord Experience
Tokyo_Q3_cleaned["host_experience"] = 2024 - Tokyo_Q3_cleaned["host_since"].dt.year

# Facility Pick-Up
important_amenities = ["Wifi", "Kitchen", "Washer", "Air conditioning", "Heating", "TV", "Elevator"]
for amenity in important_amenities:
    Tokyo_Q3_cleaned[f"has_{amenity.lower().replace(' ', '_')}"] = Tokyo_Q3_cleaned["amenities"].apply(lambda x: amenity in x)

# Calculate Distance from Centre
tokyo_center = (35.682839, 139.759455)
Tokyo_Q3_cleaned["distance_from_center"] = Tokyo_Q3_cleaned.apply(lambda row: geodesic((row["latitude"], row["longitude"]), tokyo_center).km, axis=1)

In [49]:
columns_to_drop_final = ["bathrooms_text", "host_about", "host_location", "host_verifications",
                         "host_has_profile_pic", "license", "amenities", "latitude", "longitude"]
Tokyo_Q3_cleaned = Tokyo_Q3_cleaned.drop(columns=columns_to_drop_final)
rename_dict = {
    "review_scores_communication": "review_communication_score",
    "review_scores_location": "review_location_score",
    "review_scores_value": "review_value_score",
    "host_total_listings_count": "total_listings_by_host"}
Tokyo_Q3_cleaned.rename(columns=rename_dict, inplace=True)

In [51]:
print(Tokyo_Q3_cleaned.head())

        id last_scraped                                               name  \
0   197677   2024-09-27                          Oshiage Holiday Apartment   
1   776070   2024-09-27                             Kero-kero house room 1   
2   905944   2024-09-27  4F Spacious Apartment in Shinjuku / Shibuya Tokyo   
3  1016831   2024-09-27         5 mins Shibuya Cat modern sunny  Shimokita   
4  1196177   2024-09-27        Stay with host Cozy private room Senju area   

                                         description  \
0                                                      
1  We have been in airbnb since 2011 and it has g...   
2  NEWLY RENOVATED property entirely for you & yo...   
3  Hi there, I am Wakana and I live with my two f...   
4  Ｓtay with host.We can help your travel.<br />B...   

                               neighborhood_overview  host_id  \
0                                                      964081   
1  We love Nishinippori because is nearer to Toky...   801494   

### Tokyo Q4 Data Cleaning

### Bangkok Q3 Data Cleaning